In [32]:
import pprint, random, json, time
import pandas as pd
import plotly.express as px
import pandas as pd

hierarchy = [{
		"position": "ACC{HQ}",
		"years_exp": 500,
		"positions": 11,
		"in_position": 7,
		"min_age": 52
	},
	{
		"position": "ACC",
		"years_exp": 5,
		"positions": 32,
		"in_position": 24,
		"min_age": 47
	},
	{
		"position": "RPFC-I",
		"years_exp": 5,
		"positions": 240,
		"in_position": 163,
		"min_age": 33
	},
	{
		"position": "RPFC-II",
		"years_exp": 5,
		"positions": 302,
		"in_position": 286,
		"min_age": 27
	},
	{
		"position": "APFC",
		"years_exp": 5,
		"positions": 445,
		"in_position": 243,
		"min_age": 20
	}
]

year = 2023
eid_counter = 0 
dfh = pd.DataFrame(hierarchy)
df  = pd.DataFrame(columns=['eid','desig','exp','age','active'])
dfy = pd.DataFrame(columns=['desig','eid','year','waiting'])

def drawAnimatedChart(df,col1,col2,col3, size_y=200):
  fig_bar = px.histogram(df, x=col1, y=col2, color=col1,
                 animation_frame=col3, animation_group=col1, 
                 range_y=[0,size_y],
                 color_discrete_sequence=px.colors.qualitative.T10)
  fig_bar.update_yaxes(showgrid=False),
  #fig_bar.update_xaxes(categoryorder='total descending')
  fig_bar.update_traces(hovertemplate=None)
  fig_bar.update_layout(margin=dict(t=70, b=0, l=70, r=40),
                        hovermode="x unified",
                        xaxis_tickangle=360,
                        xaxis_title=' ', yaxis_title=" ",
                        plot_bgcolor='#2d3035', paper_bgcolor='#2d3035',
                        title_font=dict(size=25, color='#a5a7ab', family="Lato, sans-serif"),
                        font=dict(color='#8a8d93'),
                        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                          )
  fig_bar.show()

def drawLineChart(df,col1,col2,col3):
  fig = px.line(df, x=col1, y=col2, color=col3)
  fig.show()

def create_eid(eid, desig, exp, age, active):
  global df
  obj = {
          'eid': [eid], 
          'desig': [desig], 
          'exp': [exp],
          'age':[age],
          'active':[active]
         }    
  df2 = pd.DataFrame(obj)
  df = pd.concat([df, df2], ignore_index = True)

def get_next_designation(desig):
    if desig == "APFC":
        return "RPFC-II"
    elif desig == "RPFC-II":
        return "RPFC-I"
    elif desig == "RPFC-I":
        return "ACC"
    elif desig == "ACC":
        return "ACC{HQ}"
    elif desig == "ACC{HQ}":
        return "ACC{HQ}"

def populate(): 
  global eid_counter
  for ind in dfh.index:
    leng = dfh['in_position'][ind]
    for pos in range(leng):
      create_eid(eid_counter, dfh['position'][ind],random.randint(0, min(dfh['years_exp'][ind],8)), random.randint(dfh['min_age'][ind],dfh['min_age'][ind]+7),True )
      eid_counter = eid_counter+1

def get_vacancy(level):
  global df, dfh
  vacancy = dfh.loc[dfh['position']==level,'positions'].iloc[0]- len(df[df['active'] & (df['desig']==level)])
  return vacancy


def get_ids(vacancy, level):
  global df
  
  df = df.sort_values(by = ['active', 'exp'], ascending = [False, False])
  filtered = df.loc[((df['desig']==level) & (df['active']==True) & (df['exp'] >= dfh.loc[dfh['position']==level, 'years_exp'].iloc[0]))]
  return filtered.head(vacancy).eid

def promote():
  global df, dfh
  
  for level in dfh['position']:
    promote_counter = 0
    next_level = get_next_designation(level)
    vacancy = int(get_vacancy(next_level))
    if vacancy>0:
      ids = get_ids(vacancy, level)
      for id in ids:
        df.loc[((df['desig']==level) & (df['active']==True) & (df['eid']==id)), 'exp'] = 0
        df.loc[((df['desig']==level) & (df['active']==True) & (df['eid']==id)), 'waiting'] = 0
        df.loc[((df['desig']==level) & (df['active']==True) & (df['eid']==id)), 'desig'] = next_level
        
      promote_counter = promote_counter+1
      #print("level:",level,"vacancy at next level:",vacancy,"promoted: ",promote_counter,"ids:",ids)
      
def hire():
  global df, dfh, eid_counter
  
  vacancy = int(get_vacancy("APFC"))
  for pos in range(vacancy):
    create_eid(eid_counter, "APFC", 0, random.randint(20,30),True )
    eid_counter = eid_counter+1



def yearChange(num=0):
  global df, dfh, year,dfy

  
  for i in range(num):
    if i==0:
      df.loc[((df['exp']>7) & (df['active']) &(df['desig'].isin(['APFC','RPFC-II','RPFC_I','ACC']))),'waiting']=1
      pivot_table = pd.pivot_table(df[df['active']], values=['eid','waiting'], index=['desig'], aggfunc={'eid':'count','waiting':'sum'}, margins=False)
      pivot_table["year"] = year
      dfyt = pivot_table[['eid','year','waiting']].reset_index()
      dfy = pd.concat([dfy, dfyt], ignore_index = True)
      print(dfy)
    df['age'] = df['age']+1
    df.loc[df['age'] <= 60, 'exp'] = df['exp']+1 
    df.loc[df['age'] > 60, 'active'] = False
    df.loc[df['age'] > 60, 'active'] = False
    if i%1==0:
      promote()
    if i%1==0:
      hire()
    df.loc[((df['exp']>7) & (df['active']) &(df['desig'].isin(['APFC','RPFC-II','RPFC_I','ACC']))),'waiting']=1
    pivot_table = pd.pivot_table(df[df['active']], values=['eid','waiting'], index=['desig'], aggfunc={'eid':'count','waiting':'sum'}, margins=False)
    pivot_table["year"] = year+1
    dfyt = pivot_table[['eid','year','waiting']].reset_index()
    dfy = pd.concat([dfy, dfyt], ignore_index = True)
    
    year = year + 1



populate()
yearChange(30)
size_y = len(dfy)
print(dfy.tail(5))
drawAnimatedChart(dfy,"desig","eid","year",900)
drawLineChart(dfy,"year","eid","desig")
drawLineChart(dfy,"year","waiting","desig")




     desig  eid  year  waiting
0      ACC   24  2023      0.0
1  ACC{HQ}    7  2023      0.0
2     APFC  243  2023      0.0
3   RPFC-I  163  2023      0.0
4  RPFC-II  286  2023      0.0
       desig  eid  year  waiting
147  RPFC-II  302  2052     60.0
148      ACC   32  2053      0.0
149     APFC  419  2053    208.0
150   RPFC-I  240  2053      0.0
151  RPFC-II  302  2053     65.0


In [31]:
from google.colab import drive
#drive.mount('drive')
dfr = df[df['active']==False]
dfr.to_csv('/content/drive/My Drive/retired.csv', encoding='utf-8', index=False)